<a href="https://colab.research.google.com/github/sanaalshboul/Gender-Voice-Detection/blob/master/Data_preperation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#import Google drive
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [ ]:
#change the current working directory to the dataset directory
#for dataset downloading: https://www.kaggle.com/liftofff9/voice-gender
import os
os.chdir('/content/drive/My Drive/pygender')
!ls

data			       magnitude_spec.png	test_data
Data_preperation.ipynb	       mfcc_features_test.csv	train_data
ELA_Voice_visualization.ipynb  mfcc_features_train.csv	Train_Test.ipynb
logs			       spectrum.png


In [ ]:
#import the required libraries
import librosa
import pickle
import pandas as pd
import numpy as np
import librosa.display
import matplotlib.pyplot as plt


from scipy.io.wavfile import read
from sklearn import preprocessing

In [ ]:
#a method for extracting a feature voice. 
#example: feature= feature_extraction("path/to/audiofie", "tonnetz")
#params
#feature_name: ["mfcc", "mel", "chroma", "tonnetz", "contrast"]
#file_name: path to the audio file
def feature_extraction(file_name, featuer_name):
  

  audio, sample_rate = librosa.core.load(file_name)
  EnrgSpectra = np.abs(librosa.stft(audio))
  result=np.asarray(())

  if featuer_name=="mfcc":
    features=np.mean(librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=25).T, axis=0)
    #rescale the data
    features=preprocessing.scale(features)
    #stack the feature arrays in a horizantal numpy array
    features=np.hstack((result,features))

  if featuer_name=="chroma":
    chroma=np.mean(librosa.feature.chroma_stft(S=EnrgSpectra, sr=sample_rate).T, axis=0)
    chroma=preprocessing.scale(chroma)
    features=np.hstack((result,chroma))
  
  if featuer_name=="mel":
    mel = np.mean(librosa.feature.melspectrogram(audio, sr=sample_rate).T, axis=0)
    mel=preprocessing.scale(mel)
    features=np.hstack((result,mel))

  if featuer_name=="tonnetz":
    tonnetz=np.mean(librosa.feature.tonnetz(y=audio,sr=sample_rate).T, axis=0)
    tonnetz=preprocessing.scale(tonnetz)
    features=np.hstack((result,tonnetz))
  if featuer_name=="contrast":
    contrast=np.mean(librosa.feature.spectral_contrast(y=audio,sr=sample_rate).T, axis=0)
    contrast=preprocessing.scale(contrast)
    features=np.hstack((result,contrast))
    
  return features

In [ ]:
#a method for extracting the feature voice for every audio file in the training or testing sets and save every feature's audio file in a speparate numpy file. 
#Then save the numpy files' paths in a csv file
#example: data_feature_preperation( "train", "chroma")
#params:
#train_or_test: ["train", "test"], feature_name: ["mfcc", "mel", "chroma", "tonnetz", "contrast"]
def data_feature_preperation(train_or_test, feature_name ):

 
  if train_or_test=="train":
    #path of the female training set
    train_female_path='/content/drive/My Drive/pygender/train_data/female/'
    #path of the male training set
    train_male_path='/content/drive/My Drive/pygender/train_data/male/'
    #path to save the feature file path of every training audio file to csv file
    csvFile='/content/drive/My Drive/pygender/mfcc_features_train.csv'
    #directory path to save the training features' files 
    dirname="data/mfcc_train"
    
  elif train_or_test=="test":
    #path of the female testing set
    train_female_path='/content/drive/My Drive/pygender/test_data/female_clips/'
    #path of the male testing set
    train_male_path='/content/drive/My Drive/pygender/test_data/male_clips/'
    #path to save the feature file path of every testing audio file to csv file
    csvFile='/content/drive/My Drive/pygender/mfcc_features_test.csv'
    #directory path to save the testing features' files 
    dirname="data/mfcc_test"
  
  #extract and save the features of female files
  female_files=[os.path.join(train_female_path,f) for f in os.listdir(train_female_path) if f.endswith('.wav')]


  if not os.path.isdir(dirname):
      os.makedirs(dirname)

  df=pd.DataFrame()
  feature_file=list()
  label=list()
  
  for f in female_files:
    splited = os.path.split(f)
    
    audio_filename=splited[-1].split(".")[0]
    
    mfcc=feature_extraction(f, feature_name)

    dest_path=f"{dirname}/{audio_filename}"
    
    np.save(dest_path, mfcc)

    feature_file.append(dest_path+".npy")
    label.append("female")

  #extract and save the features of male files
  male_files=[os.path.join(train_male_path,f) for f in os.listdir(train_male_path) if f.endswith('.wav')]

  for f in male_files: 
    mfcc=feature_extraction(f, feature_name)

    splited = os.path.split(f)
    
    audio_filename=splited[-1].split(".")[0]

    dest_path=f"{dirname}/{audio_filename}"

    np.save(dest_path, mfcc)

    feature_file.append(dest_path+".npy")
    label.append("male")

  df['file_feature_name']=feature_file
  df['gender']=label

  df.to_csv(csvFile)


In [ ]:
#method to save features of every audio file in one numpy array
#axample: save_data("test") 
#params:
#train_or_test: ["train", 'test]
def save_data(train_or_test):
  if train_or_test=="train":
    csvFile="/content/drive/My Drive/pygender/mfcc_features_train.csv"
    npFeatureFile="data/all_mfcc_train"
    npLabelFile="data/all_label_train"
  elif train_or_test=="test":
    csvFile="/content/drive/My Drive/pygender/mfcc_features_test.csv"
    npFeatureFile="data/all_mfcc_test"
    npLabelFile="data/all_label_test"
  df=pd.read_csv(csvFile)

  Y_train= np.zeros((len(df),1))
  X_train=np.zeros((len(df),25))

  for i, (file_name, gender) in enumerate(zip(df['file_feature_name'], df['gender'])):
    mfccFeature=np.load(file_name)  
    
    X_train[i]=mfccFeature

    if gender== "female":
      Y_train[i]=1
    else:
      Y_train[i]=0

  np.save(npFeatureFile, X_train)
  np.save(npLabelFile, Y_train)

  return X_train, Y_train

In [ ]:
data_feature_preperation(train_or_test="train", feature_name="mfcc")
save_data(train_or_test="train")

In [ ]:
data_feature_preperation(train_or_test="test", feature_name="mfcc")
save_data(train_or_test="test")